 ## Data Field Information

### train.csv
- The training data, comprising time series of features store_nbr, family, and onpromotion as well as the target sales.
- store_nbr identifies the store at which the products are sold.
- family identifies the type of product sold.
- sales gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
- onpromotion gives the total number of items in a product family that were being promoted at a store at a given date.

### test.csv
- The test data, having the same features as the training data. You will predict the target sales for the dates in this file.
- The dates in the test data are for the 15 days after the last date in the training data.

### stores.csv
- Store metadata, including city, state, type, and cluster.
- cluster is a grouping of similar stores.

### oil.csv
- Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)

### holidays_events.csv
- Holidays and Events, with metadata
- NOTE - Transferred column: A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
- Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).

## Additional Notes
- Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.
- A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

### Import Libraries

In [9]:
# Essential libraries for time series forecasting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Time series analysis
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Machine learning
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
import lightgbm as lgb

# To plot nice figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
from termcolor import colored

# Utility
import warnings
from datetime import datetime, timedelta
import os

# Custom function for RMSLE (competition metric)
def rmsle(y_true, y_pred):
    """Root Mean Squared Logarithmic Error"""
    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true))**2))

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('default')
pd.set_option('display.max_columns', None)
np.random.seed(42)

### Import Data

In [10]:
# Set the file path and name
train_path = './data/train.csv'
test_path = './data/test.csv'
transactions_path = './data/transactions.csv'
oil_path = './data/oil.csv'
holidays_events_path = './data/holidays_events.csv'

# Read the CSV file into a Pandas DataFrame
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
transactions_df = pd.read_csv(transactions_path)
oil_df = pd.read_csv(oil_path)
holidays_events_df = pd.read_csv(holidays_events_path)

In [11]:
# Quick overview of all datasets
print("Dataset shapes:")
print(f"Train: {train_df.shape}")
print(f"Test: {test_df.shape}")
print(f"Transactions: {transactions_df.shape}")
print(f"Oil: {oil_df.shape}")
print(f"Holidays/Events: {holidays_events_df.shape}")

Dataset shapes:
Train: (3000888, 6)
Test: (28512, 5)
Transactions: (83488, 3)
Oil: (1218, 2)
Holidays/Events: (350, 6)


### Utility Module

In [12]:
# Function to print main information about a dataset
def ds_info(df):
    print(colored('SIZE' , 'blue', attrs=['bold', 'dark']))
    print(df.size)
    print(colored('\nSHAPE' , 'blue', attrs=['bold', 'dark']))
    print(df.shape)
    print(colored('\nINFO' , 'blue', attrs=['bold', 'dark']))
    print(df.info())
    print(colored('\nNULL VALUES' , 'blue', attrs=['bold', 'dark']))
    print(df.isnull().sum())
    print(colored('\nHEAD' , 'blue', attrs=['bold', 'dark']))
    print(df.head())
    print(colored('\nTAIL' , 'blue', attrs=['bold', 'dark']))
    print(df.tail())
    print(colored('\nDESCRIBE - Numerical' , 'blue', attrs=['bold', 'dark']))
    print(df.describe())
    print(colored('\nDESCRIBE - Categorical' , 'blue', attrs=['bold', 'dark']))
    print(df.describe(include='object'))
    
    print(colored('\nUNIQUE ENTRIES IN EACH NUMERICAL COLUMN' , 'blue', attrs=['bold', 'dark']))
    # Get number of unique entries in each column with numerical data
    num_cols = [col for col in df.columns if df[col].dtype != "object"]
    num_nunique = list(map(lambda col: df[col].nunique(), num_cols))    
    dn = dict(zip(num_cols, num_nunique))
    # Print number of unique entries by column, in ascending order
    print(sorted(dn.items(), key=lambda x: x[1]))
    
    print(colored('\nUNIQUE ENTRIES IN EACH CATEGORICAL COLUMN' , 'blue', attrs=['bold', 'dark']))
    # Get number of unique entries in each column with categorical data
    object_cols = [col for col in df.columns if df[col].dtype == "object"]
    object_nunique = list(map(lambda col: df[col].nunique(), object_cols))
    dc = dict(zip(object_cols, object_nunique))
    # Print number of unique entries by column, in ascending order
    print(sorted(dc.items(), key=lambda x: x[1]))
    
    
# Function to parse date 
def parse_datetime(data):
    #convert date.datetime to type datetime 
    data.datetime = pd.to_datetime(data.datetime)
    # split in columns
    data['year'] = data['datetime'].dt.year
    data['month'] = data['datetime'].dt.month
    data['day'] = data['datetime'].dt.day
    data['hour'] = data['datetime'].dt.hour
    data['weekday'] = data['datetime'].dt.weekday
    
    
# Function to detect and print outliers 
def detect_outliers(data):
    # mean, standard deviation and 3-sigma of the data
    mean = np.mean(data)
    std = np.std(data)
    threesigma = 3 * std
    
    # print upper, lower boundary and boxplot
    sns.boxplot(data, orient="v")
    plt.show()
    lower, upper = mean-3*std, mean+3*std
    print(f"Upper and lower boundary is: {lower}/{upper}")
    
    # identify outliers and return the outliers
    outliers = [x for x in data if np.abs(x - mean) > threesigma]
    print(f"There are {len(outliers)} outliers based on three-sigma rule")

    
# Function to delete the outliers 
def delete_outliers(data, df):
    """ Detecting and dropping outliers """
    original_shape = df.shape
    mean = np.mean(data)
    std = np.std(data)
    outliers = np.abs(data-mean) > (3*std)
    outliers_num = len(train[outliers])
    df.drop(index=data[outliers].index, inplace=True)
    
    # print what was deleted
    print("Number of outliers deleted:", outliers_num)
    print ("Shape of dataframe with Ouliers: ",original_shape)
    print ("Shape of Dataframe After Deleting the Ouliers: ",df.shape)

    
# Function to plot the Correlation Heatmap
def correlation_heatmap(corr):
    """ Correlation Heatmap """
    mask = np.array(corr)
    mask[np.tril_indices_from(mask)] = False
    
    # plot the correlation
    fig,ax= plt.subplots()
    fig.set_size_inches(10,10)
    sns.heatmap(corr, mask=mask,vmax=.8, square=True,annot=True)

# Function to calculate RMSLE - Root Mean Squared Logarithmic Error   
def rmsle(y_true, y_pred):
    """ Custom RMSLE scorer function."""
    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true))**2))
    

def check_date_columns(df):
    """Identify and analyze date columns"""
    date_cols = []
    for col in df.columns:
        if df[col].dtype == 'object':
            try:
                pd.to_datetime(df[col].head(100))
                date_cols.append(col)
            except:
                pass
    print(colored('POTENTIAL DATE COLUMNS:', 'yellow', attrs=['bold']))
    print(date_cols)
    return date_cols


def missing_data_summary(df):
    """Enhanced missing data analysis"""
    missing = df.isnull().sum()
    missing_pct = 100 * missing / len(df)
    missing_table = pd.DataFrame({
        'Missing Count': missing,
        'Missing %': missing_pct
    }).sort_values('Missing %', ascending=False)
    return missing_table[missing_table['Missing Count'] > 0]


def plot_time_series(df, date_col, value_col, title="Time Series"):
    """Quick time series plot"""
    plt.figure(figsize=(12, 6))
    plt.plot(pd.to_datetime(df[date_col]), df[value_col])
    plt.title(title)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

### Dataset  Overview

In [19]:
# Quick overview of all datasets
datasets = {
    'Train': train_df,
    'Test': test_df, 
    'Transactions': transactions_df,
    'Oil': oil_df,
    'Holidays/Events': holidays_events_df
}

for name, df in datasets.items():
    print(f"{name}: {df.shape} - Columns: {list(df.columns)}")

Train: (3000888, 6) - Columns: ['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion']
Test: (28512, 5) - Columns: ['id', 'date', 'store_nbr', 'family', 'onpromotion']
Transactions: (83488, 3) - Columns: ['date', 'store_nbr', 'transactions']
Oil: (1218, 2) - Columns: ['date', 'dcoilwtico']
Holidays/Events: (350, 6) - Columns: ['date', 'type', 'locale', 'locale_name', 'description', 'transferred']


In [21]:
for name, df in datasets.items():
    print(f"\n{'='*50}")
    print(f"ANALYZING {name.upper()} DATASET")
    print('='*50)
    ds_info(df)
    print("\n" + "="*70 + "\n")


ANALYZING TRAIN DATASET
SIZE
18005328

SHAPE
(3000888, 6)

INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB
None

NULL VALUES
id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

HEAD
   id        date  store_nbr      family  sales  onpromotion
0   0  2013-01-01          1  AUTOMOTIVE    0.0            0
1   1  2013-01-01          1   BABY CARE    0.0            0
2   2  2013-01-01          1      BEAUTY    0.0            0
3   3  2013-01-01          1   BEVERAGES    0.0            0
4   4  2013-01-01          1       BOOKS    0.0            0

TAIL
              id        date  store_nbr 